# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'06-14-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'06-14-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-06-15 04:22:22,33.93911,67.709953,91458,3612,61706.0,26140.0,Afghanistan,234.939372,3.949354
1,NaN,NaN,NaN,Albania,2021-06-15 04:22:22,41.15330,20.168300,132461,2453,129865.0,143.0,Albania,4602.856349,1.851866
2,NaN,NaN,NaN,Algeria,2021-06-15 04:22:22,28.03390,1.659600,133742,3579,93094.0,37069.0,Algeria,304.991605,2.676048
3,NaN,NaN,NaN,Andorra,2021-06-15 04:22:22,42.50630,1.521800,13826,127,13618.0,81.0,Andorra,17894.260014,0.918559
4,NaN,NaN,NaN,Angola,2021-06-15 04:22:22,-11.20270,17.873900,36790,836,30359.0,5595.0,Angola,111.938477,2.272357


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/5/21,6/6/21,6/7/21,6/8/21,6/9/21,6/10/21,6/11/21,6/12/21,6/13/21,6/14/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,77963,79224,80841,82326,84050,85892,87716,88740,89861,91458
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,132374,132379,132384,132397,132415,132426,132437,132449,132459,132461
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,130681,130958,131283,131647,132034,132355,132727,133070,133388,133742


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/5/21,6/6/21,6/7/21,6/8/21,6/9/21,6/10/21,6/11/21,6/12/21,6/13/21,6/14/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,3104,3145,3187,3251,3305,3356,3412,3449,3527,3612
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2451,2451,2452,2452,2452,2452,2453,2453,2453,2453
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3510,3518,3527,3537,3544,3552,3560,3565,3571,3579


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/5/21,6/6/21,6/7/21,6/8/21,6/9/21,6/10/21,6/11/21,6/12/21,6/13/21,6/14/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,58265,58622,58998,59569,59871,60280,60598,60939,61410,61706
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,129627,129664,129702,129736,129761,129785,129807,129826,129842,129865
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,90995,91198,91413,91651,91894,92132,92384,92620,92852,93094


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/5/21,6/6/21,6/7/21,6/8/21,6/9/21,6/10/21,6/11/21,6/12/21,6/13/21,6/14/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7186,7186,7206,7206,7209,7211,7215,7215,7215,7229
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,21698,21698,21724,21744,21755,21765,21774,21774,21774,21805
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2342,2342,2343,2345,2346,2345,2345,2345,2345,2345


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/5/21,6/6/21,6/7/21,6/8/21,6/9/21,6/10/21,6/11/21,6/12/21,6/13/21,6/14/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,112,112,113,113,113,113,113,113,113,113
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,312,312,312,312,313,313,313,313,313,313
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,59,59,59,59,59,59,59,59,59,59


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
653,1001.0,Autauga,Alabama,US,2021-06-15 04:22:22,32.539527,-86.644082,7229,113,NaN,NaN,"Autauga, Alabama, US",12939.197050,1.563148
690,1075.0,Lamar,Alabama,US,2021-06-15 04:22:22,33.779950,-88.096680,1462,37,NaN,NaN,"Lamar, Alabama, US",10590.365809,2.530780
691,1077.0,Lauderdale,Alabama,US,2021-06-15 04:22:22,34.901719,-87.656247,9591,248,NaN,NaN,"Lauderdale, Alabama, US",10343.042630,2.585757


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,33474692,599945,0.0
India,29510410,374305,28162947.0
Brazil,17452612,488228,15421397.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,33474692,599945,0.0,32874747.0,2021-06-15 04:22:22,37.950547,-91.419547
India,29510410,374305,28162947.0,973158.0,2021-06-15 04:22:22,23.088275,81.806127
Brazil,17452612,488228,15421397.0,1542987.0,2021-06-15 04:22:22,-12.669522,-48.480493
France,5803012,110616,399560.0,5292836.0,2021-06-15 04:22:22,6.430808,-34.730285
Turkey,5336073,48795,5206836.0,80442.0,2021-06-15 04:22:22,38.963700,35.243300


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3803094,63204,0.0
Texas,2978351,51940,0.0
Florida,2344321,37265,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3803094,63204,0.0,3739890.0,2021-06-15 04:22:22,37.843962,-120.728594
Texas,2978351,51940,0.0,2926411.0,2021-06-15 04:22:22,31.660643,-98.653069
Florida,2344321,37265,0.0,2307056.0,2021-06-15 04:22:22,28.940755,-82.700744
New York,2109677,53558,0.0,2056119.0,2021-06-15 04:22:22,42.544151,-75.474183
Illinois,1387679,25490,0.0,1362189.0,2021-06-15 04:22:22,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1246436,24433,0.0
Illinois,Cook,555902,10427,0.0
Arizona,Maricopa,554925,10157,0.0
Florida,Miami-Dade,504650,6472,0.0
Texas,Harris,403401,6513,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1246436,24433,0.0,1222003.0,2021-06-15 04:22:22,34.308284,-118.228241,6037.0
Illinois,Cook,555902,10427,0.0,545475.0,2021-06-15 04:22:22,41.841448,-87.816588,17031.0
Arizona,Maricopa,554925,10157,0.0,544768.0,2021-06-15 04:22:22,33.348359,-112.491815,4013.0
Florida,Miami-Dade,504650,6472,0.0,498178.0,2021-06-15 04:22:22,25.611236,-80.551706,12086.0
Texas,Harris,403401,6513,0.0,396888.0,2021-06-15 04:22:22,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-12,88740,132449,133070,13813,36600,1263,4111147,223643,30248,648387,...,4574445,336470,103206,4,250309,10337,311018,6857,110332,39852
2021-06-13,89861,132459,133388,13813,36705,1263,4124190,223682,30262,648620,...,4581779,338513,103510,4,251686,10630,311534,6862,111746,39959
2021-06-14,91458,132461,133742,13826,36790,1263,4145482,223723,30273,648732,...,4589400,340818,103813,4,252883,10881,311690,6863,113134,40077


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-12,3449,2453,3565,127,825,42,85075,4482,910,10652,...,128160,4956,705,1,2814,58,3524,1347,1365,1632
2021-06-13,3527,2453,3571,127,831,42,85343,4484,910,10661,...,128168,4995,708,1,2829,59,3533,1349,1389,1632
2021-06-14,3612,2453,3579,127,836,42,86029,4488,910,10662,...,128171,5036,708,1,2845,59,3539,1350,1416,1635


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-12,60939,129826,92620,13591,30207,1214,3695288,215118,23664,633538,...,15544,296855,99152,3,230248,3827,302944,3727,96646,36992
2021-06-13,61410,129842,92852,13591,30282,1214,3721350,215195,23676,633939,...,15551,300119,99416,3,231558,3998,303813,3743,97756,37004
2021-06-14,61706,129865,93094,13618,30359,1214,3748794,215228,23692,634123,...,15562,303880,99733,3,233028,4236,304138,3765,98994,37027


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/5/21,6/6/21,6/7/21,6/8/21,6/9/21,6/10/21,6/11/21,6/12/21,6/13/21,6/14/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7186,7186,7206,7206,7209,7211,7215,7215,7215,7229
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1458,1458,1459,1459,1459,1458,1458,1458,1458,1462
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,9537,9537,9552,9556,9570,9578,9582,9582,9582,9591
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,3091,3091,3105,3108,3111,3118,3121,3121,3121,3124
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,16190,16190,16201,16203,16211,16227,16234,16234,16234,16248


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-12,547323,70652,0,886404,343873,3801738,550618,348350,109322,49,...,865085,2976974,409364,24320,3690,677671,444166,162939,676075,61114
2021-06-13,547323,70652,0,886752,343964,3801728,551091,348350,109358,49,...,865085,2977373,409558,24320,3690,677744,444166,162939,676075,61114
2021-06-14,547873,70717,0,887146,344067,3803094,551328,348465,109379,49,...,865085,2978351,409727,24331,3702,677812,444722,163144,676276,61234


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-06-12        7215   21774    2345  2673   6925    1242   2233   14692   
2021-06-13        7215   21774    2345  2673   6925    1242   2233   14692   
2021-06-14        7229   21805    2345  2676   6937    1242   2237   14710   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-06-12         3715     1872  ...    2878    670     3246      801   
2021-06-13         3715     1872  ...    2878    670     3246      801   
2021-06-14         3723     1872  ...    2882    670     3247      802   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-06-12           4636  3794  2298          0      924    656  
2021-06-13           4636  3794  2298          0      924    656  
2021-06-14           4654  3795  2301          0      924    656  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-12,11266,373,0,17761,5860,63187,6688,8261,1679,0,...,12498,51912,2323,256,28,11293,5815,2849,8013,725
2021-06-13,11266,373,0,17768,5861,63193,6689,8261,1679,0,...,12498,51929,2323,256,28,11307,5815,2849,8013,725
2021-06-14,11274,373,0,17768,5863,63204,6690,8263,1679,0,...,12498,51940,2323,256,28,11318,5782,2853,8015,725


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-06-12         113     313      59   64    139      42     71     325   
2021-06-13         113     313      59   64    139      42     71     325   
2021-06-14         113     313      59   64    139      42     71     325   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-06-12          123       45  ...      30     11       31        7   
2021-06-13          123       45  ...      30     11       31        7   
2021-06-14          123       45  ...      30     11       31        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-06-12             39    11    13          0       26      6  
2021-06-13             39    11    13          0       26      6  
2021-06-14             39    11    13          0       26      6  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-12,0.011674,0.000091,0.002584,0.000000,0.003978,0.0,0.004412,0.000394,0.000463,0.000309,...,0.001654,0.008954,0.003052,0.0,0.005984,0.028762,0.000000,0.000146,0.021839,0.004132
2021-06-13,0.012632,0.000076,0.002390,0.000000,0.002869,0.0,0.003173,0.000174,0.000463,0.000359,...,0.001603,0.006072,0.002946,0.0,0.005501,0.028345,0.001659,0.000729,0.012816,0.002685
2021-06-14,0.017772,0.000015,0.002654,0.000941,0.002316,0.0,0.005163,0.000183,0.000363,0.000173,...,0.001663,0.006809,0.002927,0.0,0.004756,0.023612,0.000501,0.000146,0.012421,0.002953


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-12,0.010844,0.0,0.001404,0.0,0.007326,0.0,0.005282,0.000000,0.0,0.000000,...,0.000094,0.010192,0.001420,0.0,0.006078,0.017544,0.000000,0.003726,0.012611,0.001842
2021-06-13,0.022615,0.0,0.001683,0.0,0.007273,0.0,0.003150,0.000446,0.0,0.000845,...,0.000062,0.007869,0.004255,0.0,0.005330,0.017241,0.002554,0.001485,0.017582,0.000000
2021-06-14,0.024100,0.0,0.002240,0.0,0.006017,0.0,0.008038,0.000892,0.0,0.000094,...,0.000023,0.008208,0.000000,0.0,0.005656,0.000000,0.001698,0.000741,0.019438,0.001838


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-12,0.005627,0.000146,0.002555,0.000000,0.003521,0.0,0.007255,0.000558,0.000465,0.000684,...,0.000064,0.012010,0.002315,0.0,0.005459,0.006046,0.000000,0.008115,0.010159,0.000595
2021-06-13,0.007729,0.000123,0.002505,0.000000,0.002483,0.0,0.007053,0.000358,0.000507,0.000633,...,0.000450,0.010995,0.002663,0.0,0.005690,0.044683,0.002869,0.004293,0.011485,0.000324
2021-06-14,0.004820,0.000177,0.002606,0.001987,0.002543,0.0,0.007375,0.000153,0.000676,0.000290,...,0.000707,0.012532,0.003189,0.0,0.006348,0.059530,0.001070,0.005878,0.012664,0.000622


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-12,0.000000,0.00000,NaN,0.000643,0.000768,0.000134,0.001098,0.00000,0.000238,0.0,...,0.0,0.000266,0.000873,0.000000,0.000000,0.000208,0.001606,0.000000,0.000000,0.000000
2021-06-13,0.000000,0.00000,NaN,0.000393,0.000265,-0.000003,0.000859,0.00000,0.000329,0.0,...,0.0,0.000134,0.000474,0.000000,0.000000,0.000108,0.000000,0.000000,0.000000,0.000000
2021-06-14,0.001005,0.00092,NaN,0.000444,0.000299,0.000359,0.000430,0.00033,0.000192,0.0,...,0.0,0.000328,0.000413,0.000452,0.003252,0.000100,0.001252,0.001258,0.000297,0.001964


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State  Alabama                                                \
Admin2          Autauga   Baldwin Barbour      Bibb    Blount Bullock   
2021-06-12      0.00000  0.000000     0.0  0.000000  0.000000     0.0   
2021-06-13      0.00000  0.000000     0.0  0.000000  0.000000     0.0   
2021-06-14      0.00194  0.001424     0.0  0.001122  0.001733     0.0   

Province_State                                         ...  Wyoming         \
Admin2            Butler   Calhoun  Chambers Cherokee  ...     Park Platte   
2021-06-12      0.000000  0.000000  0.000000      0.0  ...  0.00000    0.0   
2021-06-13      0.000000  0.000000  0.000000      0.0  ...  0.00000    0.0   
2021-06-14      0.001791  0.001225  0.002153      0.0  ...  0.00139    0.0   

Province_State                                                                \
Admin2          Sheridan  Sublette Sweetwater     Teton     Uinta Unassigned   
2021-06-12      0.000000  0.000000   0.000000  0.000000  0.000000        NaN   
2021-06-13      0.000000  0.000000   0.000000  0.000000  0.000000        NaN   
2021-06-14      0.000308  0.001248   0.003883  0.000264  0.001305        NaN   

Province_State                  
Admin2         Washakie Weston  
2021-06-12          0.0    0.0  
2021-06-13          0.0    0.0  
2021-06-14          0.0    0.0  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-12,0.00000,0.0,NaN,0.001184,0.000171,0.000285,0.001647,0.000000,0.0,NaN,...,0.0,0.000443,0.001725,0.0,0.0,0.000886,0.000000,0.000000,0.00000,0.0
2021-06-13,0.00000,0.0,NaN,0.000394,0.000171,0.000095,0.000150,0.000000,0.0,NaN,...,0.0,0.000327,0.000000,0.0,0.0,0.001240,0.000000,0.000000,0.00000,0.0
2021-06-14,0.00071,0.0,NaN,0.000000,0.000341,0.000174,0.000149,0.000242,0.0,NaN,...,0.0,0.000212,0.000000,0.0,0.0,0.000973,-0.005675,0.001404,0.00025,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-06-12         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2021-06-13         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2021-06-14         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-06-12          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2021-06-13          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2021-06-14          0.0      0.0  ...     0.0    0.0      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-06-12            0.0   0.0   0.0        NaN      0.0    0.0  
2021-06-13            0.0   0.0   0.0        NaN      0.0    0.0  
2021-06-14            0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-12,0.016349,0.000089,0.002645,0.000350,0.004061,2.433147e-06,0.005602,0.000390,0.000345,0.000354,...,0.001642,0.009864,0.002782,3.700743e-17,0.005297,0.026706,0.000254,0.001084,0.020058,0.003758
2021-06-13,0.014491,0.000082,0.002517,0.000175,0.003465,1.216574e-06,0.004387,0.000282,0.000404,0.000357,...,0.001622,0.007968,0.002864,1.850372e-17,0.005399,0.027525,0.000956,0.000907,0.016437,0.003221
2021-06-14,0.016131,0.000049,0.002586,0.000558,0.002890,6.082868e-07,0.004775,0.000233,0.000384,0.000265,...,0.001643,0.007389,0.002895,9.251859e-18,0.005078,0.025569,0.000729,0.000526,0.014429,0.003087


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-12,0.013586,0.000108,0.001833,3.759863e-15,0.006388,2.090184e-09,0.006647,0.000518,2.385483e-22,0.000093,...,0.000094,0.010196,0.001929,0.0,0.005897,0.019291,0.000179,0.002859,0.010498,0.002059
2021-06-13,0.018101,0.000054,0.001758,1.879931e-15,0.006830,1.045092e-09,0.004899,0.000482,1.192742e-22,0.000469,...,0.000078,0.009033,0.003092,0.0,0.005614,0.018266,0.001367,0.002172,0.014040,0.001029
2021-06-14,0.021100,0.000027,0.001999,9.399657e-16,0.006424,5.225461e-10,0.006468,0.000687,5.963708e-23,0.000281,...,0.000051,0.008620,0.001546,0.0,0.005635,0.009133,0.001532,0.001457,0.016739,0.001434


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-12,0.005769,0.000168,0.002606,0.000416,0.005377,2.402468e-06,0.007599,0.000601,0.000383,0.000685,...,0.000073,0.012108,0.002537,8.881784e-16,0.005531,0.014862,0.000301,0.006333,0.008278,0.000877
2021-06-13,0.006749,0.000146,0.002556,0.000208,0.003930,1.201234e-06,0.007326,0.000479,0.000445,0.000659,...,0.000262,0.011552,0.002600,4.440892e-16,0.005610,0.029773,0.001585,0.005313,0.009882,0.000601
2021-06-14,0.005785,0.000161,0.002581,0.001097,0.003236,6.006171e-07,0.007350,0.000316,0.000560,0.000475,...,0.000484,0.012042,0.002894,2.220446e-16,0.005979,0.044651,0.001327,0.005595,0.011273,0.000611


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-12,0.000224,0.000301,NaN,0.000552,0.000843,0.000223,0.001247,0.000078,0.000285,1.463639e-137,...,0.000122,0.000501,0.000786,0.000184,0.004007,0.000216,0.001531,0.000322,0.000103,0.000739
2021-06-13,0.000112,0.000151,NaN,0.000473,0.000554,0.000110,0.001053,0.000039,0.000307,7.318197e-138,...,0.000061,0.000317,0.000630,0.000092,0.002003,0.000162,0.000766,0.000161,0.000052,0.000369
2021-06-14,0.000558,0.000535,NaN,0.000458,0.000427,0.000235,0.000742,0.000185,0.000249,3.659098e-138,...,0.000030,0.000323,0.000521,0.000272,0.002628,0.000131,0.001009,0.000710,0.000174,0.001166


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-06-12      0.000249  0.000242  0.000008  0.000288  0.000208  0.000142   
2021-06-13      0.000125  0.000121  0.000004  0.000144  0.000104  0.000071   
2021-06-14      0.001033  0.000772  0.000002  0.000633  0.000919  0.000035   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-06-12      0.000422  0.000094  0.000251  0.000193  ...  0.000254   
2021-06-13      0.000211  0.000047  0.000125  0.000097  ...  0.000127   
2021-06-14      0.001001  0.000636  0.001139  0.000048  ...  0.000758   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-06-12      0.001220  0.000401  0.000548   0.001188  0.000116  0.000450   
2021-06-13      0.000610  0.000200  0.000274   0.000594  0.000058  0.000225   
2021-06-14      0.000305  0.000254  0.000761   0.002238  0.000161  0.000765   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-06-12           -1.0 -0.000084  0.000048  
2021-06-13           -1.0 -0.000042  0.000024  
2021-06-14           -1.0 -0.000021  0.000012  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-12,0.000545,0.002710,NaN,0.000794,0.000305,0.000429,0.001732,0.000112,0.000166,NaN,...,0.000108,0.000589,0.001029,0.000008,0.000018,0.000926,0.000504,0.000770,0.000582,0.000217
2021-06-13,0.000273,0.001355,NaN,0.000594,0.000238,0.000262,0.000941,0.000056,0.000083,NaN,...,0.000054,0.000458,0.000514,0.000004,0.000009,0.001083,0.000252,0.000385,0.000291,0.000109
2021-06-14,0.000491,0.000678,NaN,0.000297,0.000289,0.000218,0.000545,0.000149,0.000041,NaN,...,0.000027,0.000335,0.000257,0.000002,0.000005,0.001028,-0.002711,0.000895,0.000270,0.000054


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                      \
Admin2           Autauga   Baldwin       Barbour          Bibb        Blount   
2021-06-12      0.000159  0.000204  2.652073e-07  1.157245e-13  1.366314e-11   
2021-06-13      0.000080  0.000102  1.326036e-07  5.786224e-14  6.831570e-12   
2021-06-14      0.000040  0.000051  6.630182e-08  2.893112e-14  3.415785e-12   

Province_State                                                      \
Admin2               Bullock        Butler       Calhoun  Chambers   
2021-06-12      4.543321e-11  1.124467e-07  3.068373e-06 -0.002012   
2021-06-13      2.271661e-11  5.622335e-08  1.534186e-06 -0.001006   
2021-06-14      1.135830e-11  2.811167e-08  7.670932e-07 -0.000503   

Province_State                ...       Wyoming                              \
Admin2              Cherokee  ...          Park        Platte      Sheridan   
2021-06-12      1.262878e-18  ...  7.131806e-27  2.894773e-43  5.386386e-29   
2021-06-13      6.314389e-19  ...  3.565903e-27  1.447386e-43  2.693193e-29   
2021-06-14      3.157194e-19  ...  1.782952e-27  7.236932e-44  1.346597e-29   

Province_State                                                                 \
Admin2              Sublette    Sweetwater     Teton         Uinta Unassigned   
2021-06-12      2.366288e-49  7.842962e-10  0.013889  7.579123e-14   0.752941   
2021-06-13      1.183144e-49  3.921481e-10  0.006944  3.789561e-14   0.752941   
2021-06-14      5.915720e-50  1.960741e-10  0.003472  1.894781e-14   0.752941   

Province_State                          
Admin2              Washakie    Weston  
2021-06-12      1.470214e-41  0.012500  
2021-06-13      7.351072e-42  0.006250  
2021-06-14      3.675536e-42  0.003125  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
